In [20]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.join(os.getcwd(), ".."))

from models.sp500.fundamentals import FundamentalStockAnalyzer
import logging
import yfinance as yf
import time

logging.basicConfig(level=logging.CRITICAL)  # or DEBUG for more detail

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# Create analyzer using the new async factory method
async def setup_analyzer():
    # Use the async factory method instead of direct constructor
    fas = await FundamentalStockAnalyzer.create()

    # Get tickers (now async)
    tickers = await fas.tickers
    print(f"Loaded {len(tickers)} S&P 500 tickers")
    print(f"First 10 tickers: {tickers[:10]}")

    # Show available field sets (this is still synchronous)
    print(f"\nAvailable field sets: {fas.get_available_field_sets()}")

    return fas


# Run the async setup
fas = await setup_analyzer()

Loaded 503 S&P 500 tickers
First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

Available field sets: ['quick_overview', 'value_analysis', 'fundamental_analysis', 'price_performance', 'complete', 'basic_info', 'valuation_metrics', 'profitability_metrics', 'cash_flow_metrics', 'growth_metrics', 'percentage_changes', 'financial_health', 'dividend_info', 'analyst_data', 'earnings_data', 'all']


In [ ]:
fas = await FundamentalStockAnalyzer.create()
tickers = await fas.tickers
print(f"Loaded {len(tickers)} S&P 500 tickers")
print(f"First 10 tickers: {tickers[:10]}")

# Show available field sets (this is still synchronous)
print(f"\nAvailable field sets: {fas.get_available_field_sets()}")

In [ ]:
# Test single ticker fundamental data (now async)
print("=== Single Ticker Example ===")
result = await fas.get_fundamentals("AAPL", field_set="basic_info")
print(f"AAPL basic info: {result}")

print("\n=== Error Handling Example ===")
error_result = await fas.get_fundamentals("INVALID_TICKER")
print(f"Invalid ticker result: {error_result}")

In [ ]:
# Test multiple tickers (small sample) - now with async and concurrency control
print("=== Multiple Tickers Example ===")
sample_tickers = ["AAPL", "MSFT", "GOOGL"]

start_time = time.time()

results = await fas.get_fundamentals_concurrent(
    tickers=sample_tickers,
    field_set="basic_info",  # Using basic_info since value_analysis might not exist
    max_concurrent=3,  # Process all 3 concurrently
)

end_time = time.time()
print(f"Fetched {len(sample_tickers)} tickers in {end_time - start_time:.2f} seconds")

for result in results:
    if "error" not in result:
        ticker = result["ticker"]
        data = result["data"]
        short_name = data.get("shortName", "Unknown")
        market_cap = data.get("marketCap", "N/A")
        print(f"{ticker}: {short_name} - Market Cap: {market_cap}")
    else:
        print(f"{result['ticker']}: Error - {result['error']}")

print("\n=== Configuration Examples ===")
print(f"Cache days setting: {fas.config.cache_days}")
print(f"Data source URL: {fas.config.data_url}")

# Show different field sets
basic_fields = fas.get_fields("basic_info")
all_field_sets = fas.get_available_field_sets()
print(f"Basic info fields ({len(basic_fields)}): {basic_fields}")
print(f"All available field sets: {all_field_sets}")
if "value_analysis" in all_field_sets:
    value_fields = fas.get_fields("value_analysis")
    print(f"Value analysis fields ({len(value_fields)}): {value_fields}")

In [ ]:
stock = yf.Ticker("V")
hist = stock.history(period="1y")
hist.head()

In [ ]:
# Performance comparison: Sequential vs Concurrent processing
print("=== Performance Comparison ===")

# Test with more tickers to see the performance difference
test_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NFLX", "NVDA"]

print(f"Testing with {len(test_tickers)} tickers: {', '.join(test_tickers)}")

# Sequential processing (max_concurrent=1)
print("\n1. Sequential processing:")
start = time.time()
sequential_results = await fas.get_fundamentals_concurrent(
    tickers=test_tickers, field_set="basic_info", max_concurrent=1
)
sequential_time = time.time() - start
print(f"   Time: {sequential_time:.2f} seconds")

# Concurrent processing (max_concurrent=5)
print("\n2. Concurrent processing:")
start = time.time()
concurrent_results = await fas.get_fundamentals_concurrent(
    tickers=test_tickers, field_set="basic_info", max_concurrent=5
)
concurrent_time = time.time() - start
print(f"   Time: {concurrent_time:.2f} seconds")

# Calculate speedup
if concurrent_time > 0:
    speedup = sequential_time / concurrent_time
    print(f"\n🚀 Speedup: {speedup:.1f}x faster with concurrency!")
    print(f"   Time saved: {sequential_time - concurrent_time:.1f} seconds")

# Show some results
print(f"\nSuccessfully fetched data for {len(concurrent_results)} tickers:")
for result in concurrent_results[:3]:  # Show first 3
    if "error" not in result:
        ticker = result["ticker"]
        market_cap = result["data"].get("marketCap", "N/A")
        print(f"  {ticker}: Market Cap = {market_cap}")
print("  ...")

In [ ]:
# Fetch valuation metrics for the test tickers sequentially (no concurrency)
# This uses the "valuation_metrics" field set, which is available in all_field_sets

sequential_results = await fas.get_fundamentals_concurrent(
    tickers=test_tickers,
    field_set="cash_flow_metrics",
)

# Display results in a user-friendly format
for result in sequential_results:
    ticker = result["ticker"]
    data = result.get("data", {})
    if "error" in result:
        print(f"{ticker}: Error - {result['error']}")
    else:
        print(f"\n{ticker} Valuation Metrics:")
        for field, value in data.items():
            print(f"  {field}: {value}")

In [ ]:
# Recreate analyzer to pick up config changes
print("=== Recreating analyzer with updated config ===")
fas = await FundamentalStockAnalyzer.create()

# Check available field sets
print(f"Available field sets: {fas.get_available_field_sets()}")

# Check if our new percentage_changes field group is available
try:
    percentage_fields = fas.get_fields("percentage_changes")
    print(f"Percentage change fields: {percentage_fields}")
except ValueError as e:
    print(f"Error: {e}")

# Check if price_performance field set is available
try:
    price_performance_fields = fas.get_fields("price_performance")
    print(f"Price performance fields: {price_performance_fields}")
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
# Test the percentage change calculations
print("=== Testing Percentage Change Calculations ===")

# Test with AAPL
print("Testing AAPL percentage changes:")
result = await fas.get_fundamentals("AAPL", field_set="percentage_changes")
if "error" not in result:
    for field, value in result["data"].items():
        print(f"  {field}: {value}%")
else:
    print(f"Error: {result['error']}")

print("\n" + "=" * 50)

# Test with price_performance field set for MSFT
print("Testing MSFT price performance:")
result = await fas.get_fundamentals("MSFT", field_set="price_performance")
if "error" not in result:
    for field, value in result["data"].items():
        if field == "currentPrice":
            print(f"  {field}: ${value}")
        else:
            print(f"  {field}: {value}%")
else:
    print(f"Error: {result['error']}")

print("\n" + "=" * 50)

# Test multiple tickers with percentage changes
print("Testing multiple tickers with percentage changes:")
sample_tickers = ["AAPL", "GOOGL"]
results = await fas.get_fundamentals_concurrent(
    tickers=sample_tickers, field_set="percentage_changes", max_concurrent=2
)

for result in results:
    ticker = result["ticker"]
    if "error" not in result:
        print(f"\n{ticker}:")
        annual = result["data"].get("annualChangePercent")
        one_week = result["data"].get("oneWeekChangePercent")
        print(f"  Annual Change: {annual}%")
        print(f"  One Week Change: {one_week}%")
    else:
        print(f"{ticker}: Error - {result['error']}")

In [ ]:
# Final integration test - combining traditional fundamentals with percentage changes
print("=== Integration Test: Traditional + Percentage Changes ===")

# Test mixing basic info with percentage changes
result = await fas.get_fundamentals("TSLA", field_set="basic_info")
print("TSLA basic info (without percentage changes):")
for field, value in result["data"].items():
    print(f"  {field}: {value}")

print("\n" + "-" * 40)

# Now test basic_info + specific percentage fields by using "all" field set
result = await fas.get_fundamentals("TSLA", field_set="all")
print("TSLA sample of all fields (showing some basics + percentage changes):")
data = result["data"]
sample_fields = [
    "shortName",
    "currentPrice",
    "marketCap",
    "annualChangePercent",
    "sixMonthChangePercent",
    "oneWeekChangePercent",
]

for field in sample_fields:
    value = data.get(field)
    if field in [
        "annualChangePercent",
        "sixMonthChangePercent",
        "oneWeekChangePercent",
    ]:
        print(f"  {field}: {value}%")
    elif field == "currentPrice":
        print(f"  {field}: ${value}")
    else:
        print(f"  {field}: {value}")

print(
    f"\n✅ Successfully integrated {len([f for f in data.keys() if 'ChangePercent' in f])} percentage change fields!"
)
print("✅ All percentage change calculations completed successfully!")
print("✅ Configuration system updated with new field groups!")
print("✅ Async implementation maintains performance!")

## Stock Filtering with Custom Logic in S&P 500 Fundamentals Model

In [ ]:
fas = await FundamentalStockAnalyzer.create()
print("=== Fetching full S&P 500 fundamentals concurrently ===")
sp500_data = await fas.get_fundamentals_concurrent(field_set="all", max_concurrent=10)
print(f"Fetched data for {len(sp500_data)} S&P 500 companies.")

### Strict Criteria for Stock Selection
**Selection Criteria:**

- Annual Change ≤ **-25%**
- Six Month Change ≤ **0%**
- Three Month Change ≤ **0%**
- One Month Change ≤ **0%**
- One Week Change ≤ **0%**
- Cash Flow > **$300,000,000**
- Free Cash Flow > **$300,000,000**
- EBITDA > **0**
- Profit Margins > **0**
- Revenue Growth > **-10%**
- EBITDA Margins > **0**
- Operating Margins > **0**
- Return on Assets > **0**

These strict filters are designed to identify S&P 500 stocks with significant recent declines across multiple timeframes, while ensuring the company maintains strong positive cash flow. This approach helps surface potential turnaround candidates with robust financial health.


In [ ]:
# Get list of stocks that fit the strict criteria and display them
print("=== Filtering stocks that fit the strict criteria ===")
filtered_stocks = [
    stock
    for stock in sp500_data
    if stock["data"].get("annualChangePercent") is not None
    and stock["data"]["annualChangePercent"] <= -25
    and stock["data"].get("sixMonthChangePercent") is not None
    and stock["data"]["sixMonthChangePercent"] <= 0
    and stock["data"].get("threeMonthChangePercent") is not None
    and stock["data"]["threeMonthChangePercent"] <= 0
    and stock["data"].get("oneMonthChangePercent") is not None
    and stock["data"]["oneMonthChangePercent"] <= 0
    and stock["data"].get("oneWeekChangePercent") is not None
    and stock["data"]["oneWeekChangePercent"] <= 0
    and stock["data"].get("operatingCashflow") is not None
    and stock["data"]["operatingCashflow"] > 300000000
    and stock["data"].get("freeCashflow") is not None
    and stock["data"]["freeCashflow"] > 300000000
    and stock["data"].get("ebitda") is not None
    and stock["data"]["ebitda"] > 0
    and stock["data"].get("profitMargins") is not None
    and stock["data"]["profitMargins"] > 0
    and stock["data"].get("revenueGrowth") is not None
    and stock["data"]["revenueGrowth"] > -10
    and stock["data"].get("ebitdaMargins") is not None
    and stock["data"]["ebitdaMargins"] > 0
    and stock["data"].get("operatingMargins") is not None
    and stock["data"]["operatingMargins"] > 0
    and stock["data"].get("returnOnAssets") is not None
    and stock["data"]["returnOnAssets"] > 0
]
print(f"Found {len(filtered_stocks)} stocks that fit the strict criteria:")
for stock in filtered_stocks:
    ticker = stock["ticker"]
    name = stock["data"].get("shortName", "N/A")
    annual_change = stock["data"].get("annualChangePercent", "N/A")
    six_month_change = stock["data"].get("sixMonthChangePercent", "N/A")
    three_month_change = stock["data"].get("threeMonthChangePercent", "N/A")
    one_month_change = stock["data"].get("oneMonthChangePercent", "N/A")
    one_week_change = stock["data"].get("oneWeekChangePercent", "N/A")
    cash_flow = stock["data"].get("cashFlow", "N/A")
    free_cash_flow = stock["data"].get("freeCashFlow", "N/A")
    ebitda = stock["data"].get("ebitda", "N/A")
    profit_margins = stock["data"].get("profitMargins", "N/A")
    revenue_growth = stock["data"].get("revenueGrowth", "N/A")
    ebitda_margins = stock["data"].get("ebitdaMargins", "N/A")
    operating_margins = stock["data"].get("operatingMargins", "N/A")
    return_on_assets = stock["data"].get("returnOnAssets", "N/A")

    print(f"\nTicker: {ticker} - {name}")
    print(f"  Annual Change: {annual_change}%")
    print(f"  6-Month Change: {six_month_change}%")
    print(f"  3-Month Change: {three_month_change}%")
    print(f"  1-Month Change: {one_month_change}%")
    print(f"  1-Week Change: {one_week_change}%")
    print(f"  Cash Flow: ${cash_flow}")
    print(f"  Free Cash Flow: ${free_cash_flow}")
    print(f"  EBITDA: ${ebitda}")
    print(f"  Profit Margins: {profit_margins}")
    print(f"  Revenue Growth: {revenue_growth}%")
    print(f"  EBITDA Margins: {ebitda_margins}")
    print(f"  Operating Margins: {operating_margins}")
    print(f"  Return on Assets: {return_on_assets}%")

### Relaxed Criteria for Stock Selection
**Selection Criteria:**
Exactly one of the following fields must be between 0 and 3% and the others must be less than or equal to 0%:
- Six Month Change
- Three Month Change
- One Month Change
- One Week Change

All other criteria remain the same:
- Annual Change ≤ **-25%**
- Cash Flow > **$300,000,000**
- Free Cash Flow > **$300,000,000**
- EBITDA > **0**
- Profit Margins > **0**
- Revenue Growth > **-10%**
- EBITDA Margins > **0**
- Operating Margins > **0**
- Return on Assets > **0**

This relaxed filter aims to identify stocks that have experienced slight positive movements in one of the shorter-term timeframes, while still showing overall weakness in the others. This can help find stocks that may be starting to stabilize or recover after a period of decline.



In [ ]:
# Modified version that captures BOTH scenarios:
# 1. Exactly one short-term field in (0, 3%] and others <= 0 (original logic)
# 2. ALL short-term fields <= 0 (new addition)

relaxed_filtered_stocks = []
all_negative_stocks = []  # New: track stocks with all periods <= 0

for i, stock in enumerate(sp500_data, start=1):
    data = stock["data"]
    # Check annual change and financial criteria (same as before)
    if (
        data.get("annualChangePercent") is not None
        and data["annualChangePercent"] <= -25
        and data.get("operatingCashflow") is not None
        and data["operatingCashflow"] > 300000000
        and data.get("freeCashflow") is not None
        and data["freeCashflow"] > 300000000
        and data.get("ebitda") is not None
        and data["ebitda"] > 0
        and data.get("profitMargins") is not None
        and data["profitMargins"] > 0
        and data.get("revenueGrowth") is not None
        and data["revenueGrowth"] > -10
        and data.get("ebitdaMargins") is not None
        and data["ebitdaMargins"] > 0
        and data.get("operatingMargins") is not None
        and data["operatingMargins"] > 0
        and data.get("returnOnAssets") is not None
        and data["returnOnAssets"] > 0
    ):
        # Check short-term change criteria
        change_fields = [
            "sixMonthChangePercent",
            "threeMonthChangePercent",
            "oneMonthChangePercent",
            "oneWeekChangePercent",
        ]
        values = []
        valid = True
        for field in change_fields:
            val = data.get(field)
            if val is None:
                valid = False
                break
            values.append(val)

        if valid:
            # Original logic: exactly one in (0, 3] and others <= 0
            in_range = [0 < v <= 3 for v in values]
            others_nonpos = [v <= 0 for i, v in enumerate(values) if not in_range[i]]

            # NEW: Check if ALL short-term periods are <= 0
            all_nonpositive = all(v <= 0 for v in values)

            if sum(in_range) == 1 and all(others_nonpos):
                relaxed_filtered_stocks.append(stock)
            elif all_nonpositive:  # NEW condition
                all_negative_stocks.append(stock)

# Display results
print(
    f"🎯 Stocks with exactly one positive short-term period (0-3%): {len(relaxed_filtered_stocks)}"
)
print(f"📉 Stocks with ALL short-term periods <= 0%: {len(all_negative_stocks)}")
print(
    f"🔢 Total qualifying stocks: {len(relaxed_filtered_stocks) + len(all_negative_stocks)}"
)

# Show stocks with one positive period
if relaxed_filtered_stocks:
    print("\n=== Stocks with one positive short-term period (0-3%) ===")
    for stock in relaxed_filtered_stocks:
        ticker = stock["ticker"]
        name = stock["data"].get("shortName", "N/A")
        annual = stock["data"].get("annualChangePercent", "N/A")
        six_month = stock["data"].get("sixMonthChangePercent", "N/A")
        three_month = stock["data"].get("threeMonthChangePercent", "N/A")
        one_month = stock["data"].get("oneMonthChangePercent", "N/A")
        one_week = stock["data"].get("oneWeekChangePercent", "N/A")

        print(f"   {ticker} - {name}")
        print(
            f"     Annual: {annual}%, 6M: {six_month}%, 3M: {three_month}%, 1M: {one_month}%, 1W: {one_week}%"
        )

# Show all negative stocks
if all_negative_stocks:
    print("\n=== Stocks with ALL short-term periods <= 0% ===")
    for stock in all_negative_stocks:
        ticker = stock["ticker"]
        name = stock["data"].get("shortName", "N/A")
        annual = stock["data"].get("annualChangePercent", "N/A")
        six_month = stock["data"].get("sixMonthChangePercent", "N/A")
        three_month = stock["data"].get("threeMonthChangePercent", "N/A")
        one_month = stock["data"].get("oneMonthChangePercent", "N/A")
        one_week = stock["data"].get("oneWeekChangePercent", "N/A")

        print(f"   {ticker} - {name}")
        print(
            f"     Annual: {annual}%, 6M: {six_month}%, 3M: {three_month}%, 1M: {one_month}%, 1W: {one_week}%"
        )

if len(relaxed_filtered_stocks) == 0 and len(all_negative_stocks) == 0:
    print("😢 No stocks found that meet either criteria.")

# Recommended Stocks Method Implementation

The `recommended_stocks()` method is part of the `FundamentalStockAnalyzer` class.

### Key Features:

1. **Two Criteria Categories:**
   - **Signs of Recovery:** Stocks with exactly one short-term period (6M, 3M, 1M, 1W) showing modest gains (0-3%) while others are ≤ 0%
   - **Full Decline:** Stocks with all short-term periods ≤ 0%

2. **Strong Financial Requirements:** 
   - Annual decline ≥ 25%
   - Operating cash flow > $300M
   - Free cash flow > $300M  
   - Positive EBITDA, profit margins, EBITDA margins, operating margins, and ROA
   - Revenue growth > -10%

3. **Usage:**
   ```python
   # Get recommendations for all S&P 500 stocks
   analyzer = FundamentalStockAnalyzer()
   recommendations = await analyzer.recommended_stocks()
   
   # Get recommendations for specific tickers
   subset_recommendations = await analyzer.recommended_stocks(
       tickers=["AAPL", "MSFT", "GOOGL"], 
       max_concurrent=5
   )
   ```

4. **Performance:** Uses async/await with configurable concurrency for efficient data fetching

The method returns a simple list of ticker symbols, making it easy to integrate into trading systems, portfolio analysis, or further research workflows.

In [ ]:
# Test the new recommended_stocks method
print("=== Testing recommended_stocks method ===")

# Create a new analyzer instance
fas_new = await FundamentalStockAnalyzer.create()

# Get recommendations using the new method
print("Fetching stock recommendations...")
start_time = time.time()
recommended_tickers = await fas_new.recommended_stocks(max_concurrent=10)
end_time = time.time()

print(f"✅ Analysis completed in {end_time - start_time:.2f} seconds")
print(f"📈 Total recommended stocks: {len(recommended_tickers)}")
print(f"🎯 Recommended tickers: {recommended_tickers}")

# Compare with manual filtering results
manual_results = set(
    [stock["ticker"] for stock in relaxed_filtered_stocks + all_negative_stocks]
)
method_results = set(recommended_tickers)

print("\n=== Comparison with Manual Filtering ===")
print(f"Manual filtering found: {len(manual_results)} stocks")
print(f"Method filtering found: {len(method_results)} stocks")
print(f"Results match: {manual_results == method_results}")

if manual_results != method_results:
    print(f"In manual but not method: {manual_results - method_results}")
    print(f"In method but not manual: {method_results - manual_results}")
else:
    print("🎉 Perfect match! The method implementation is correct.")

In [ ]:
# Example usage with a subset of tickers for faster testing
print("=== Example: Testing with a subset of popular stocks ===")

# Test with some popular tickers that might be in different categories
popular_tickers = [
    "AAPL",
    "MSFT",
    "GOOGL",
    "AMZN",
    "TSLA",
    "META",
    "NFLX",
    "NVDA",
    "V",
    "JPM",
]

subset_recommendations = await fas_new.recommended_stocks(
    tickers=popular_tickers, max_concurrent=5
)

print(f"Analyzed {len(popular_tickers)} popular stocks")
print(f"Found {len(subset_recommendations)} recommendations: {subset_recommendations}")

if subset_recommendations:
    print("\n=== Details for subset recommendations ===")
    # Get details for recommended stocks
    details = await fas_new.get_fundamentals_concurrent(
        tickers=subset_recommendations, field_set="all", max_concurrent=3
    )

    for stock in details:
        if "error" not in stock:
            ticker = stock["ticker"]
            data = stock["data"]
            name = data.get("shortName", "N/A")
            annual = data.get("annualChangePercent", "N/A")
            six_month = data.get("sixMonthChangePercent", "N/A")
            three_month = data.get("threeMonthChangePercent", "N/A")
            one_month = data.get("oneMonthChangePercent", "N/A")
            one_week = data.get("oneWeekChangePercent", "N/A")

            print(f"\n   {ticker} - {name}")
            print(
                f"     Annual: {annual}%, 6M: {six_month}%, 3M: {three_month}%, 1M: {one_month}%, 1W: {one_week}%"
            )
else:
    print(
        "No recommendations found in this subset (expected for popular/stable stocks)"
    )

## Scoring System for Recommended Stocks

### Methodology

The scoring system uses a **ranking-based approach** to compare recommended stocks across multiple financial metrics. This method ensures fair comparison regardless of the absolute values or units of different metrics.

### Scoring Process

1. **Data Preparation**: Creates a pandas DataFrame containing only the recommended stocks with relevant financial metrics

2. **Ranking System**: For each scoring criterion, stocks are ranked from **best to worst**:
   - Uses pandas `rank(ascending=True, method="max")` 
   - **Lower rank number = Better performance** (Rank 1 = Best stock for that metric)
   - `method="max"` ensures tied values get the higher rank number

3. **Score Aggregation**: Individual ranking scores are summed to create a `total_score`

4. **Final Ranking**: Stocks are sorted by total score in **descending order** (highest total score = best overall)

### Scoring Criteria (9 Financial Metrics)

- **Operating Cashflow**: Higher cash flow scores better (lower rank number)
- **Free Cashflow**: Higher free cash flow scores better  
- **EBITDA**: Higher EBITDA scores better
- **Profit Margins**: Higher profit margins score better
- **Revenue Growth**: Less negative or positive growth scores better
- **EBITDA Margins**: Higher EBITDA margins score better
- **Operating Margins**: Higher operating margins score better
- **Return on Assets**: Higher return on assets scores better
- **Return on Equity**: Higher return on equity scores better

### Interpretation

- **Best possible total score**: 9 (if a stock ranks #1 in all 9 metrics)
- **Worst possible total score**: N × 9 (where N = number of recommended stocks)
- **Lower total scores indicate better overall financial performance**
- The system provides a balanced view across multiple fundamental analysis dimensions

This ranking approach eliminates the need to weight different metrics and handles varying scales naturally, making it ideal for comparing stocks with different market capitalizations and business models.

In [ ]:
# Get data for recommended stocks into a pandas DF and add the fields used for the scoring criteria
import pandas as pd

df = pd.DataFrame(
    [stock["data"] for stock in sp500_data if "data" in stock],
    index=[stock["ticker"] for stock in sp500_data if "data" in stock],
)

# Keep only rows for the reccomended stocks
df = df.loc[recommended_tickers]
df = df[
    [
        "shortName",
        "annualChangePercent",
        "sixMonthChangePercent",
        "threeMonthChangePercent",
        "oneMonthChangePercent",
        "oneWeekChangePercent",
        "operatingCashflow",
        "freeCashflow",
        "ebitda",
        "profitMargins",
        "revenueGrowth",
        "ebitdaMargins",
        "operatingMargins",
        "returnOnAssets",
        "returnOnEquity",
    ]
]

columns_to_score = [
    "operatingCashflow",
    "freeCashflow",
    "ebitda",
    "profitMargins",
    "revenueGrowth",
    "ebitdaMargins",
    "operatingMargins",
    "returnOnAssets",
    "returnOnEquity",
]

# Rank and score each column (1 = best)
for column in columns_to_score:
    df.loc[:, f"{column}_score"] = (
        df[column].rank(ascending=True, method="max").astype(int)
    )

# Sum the scores to get a total score
df.loc[:, "total_score"] = df[[f"{col}_score" for col in columns_to_score]].sum(axis=1)

# Sort by total score (highest is best)
df = df.sort_values("total_score", ascending=False)
df.head(len(df))

## Testing the score_recommended_stocks() Method

The `score_recommended_stocks()` method in `FundamentalStockAnalyzer` implements the scoring logic from the notebook above as a reusable class method.

### Key Features:

1. **Integrated Workflow**: Automatically gets recommended stocks and scores them in one method call
2. **Async Implementation**: Uses async/await for efficient data fetching
3. **Robust Data Handling**: Handles missing data and type conversion safely
4. **Structured Output**: Returns sorted list of dictionaries with comprehensive stock data

### Method Signature:
```python
async def score_recommended_stocks(
    self,
    tickers: Optional[List[str]] = None,
    max_concurrent: int = 10,
) -> List[Dict[str, Any]]
```

### Return Format:
Each stock in the returned list contains:
- `ticker`: Stock symbol
- `total_score`: Aggregate ranking score (lower = better)
- `shortName`: Company name  
- All individual scoring metrics and their rank scores
- Performance data (price changes)
- Financial metrics used in scoring

The list is sorted by `total_score` in ascending order (best performers first).

In [ ]:
# Test the new score_recommended_stocks method
print("=== Testing score_recommended_stocks method ===")

# Create a new analyzer instance
fas_scoring = await FundamentalStockAnalyzer.create()

# Get scored recommendations
print("Fetching and scoring recommended stocks...")
start_time = time.time()
scored_stocks = await fas_scoring.score_recommended_stocks(max_concurrent=10)
end_time = time.time()

print(f"✅ Scoring completed in {end_time - start_time:.2f} seconds")
print(f"📊 Total scored stocks: {len(scored_stocks)}")

if scored_stocks:
    print("\n=== Top 10 Recommended Stocks (Best Scores) ===")
    for i, stock in enumerate(scored_stocks[:10], 1):
        ticker = stock["ticker"]
        name = stock.get("shortName", "N/A")
        total_score = stock["total_score"]
        annual_change = stock.get("annualChangePercent", "N/A")

        # Show some key financial metrics
        operating_cf = stock.get("operatingCashflow", "N/A")
        profit_margins = stock.get("profitMargins", "N/A")
        roa = stock.get("returnOnAssets", "N/A")

        print(f"{i:2d}. {ticker} - {name}")
        print(f"     Total Score: {total_score} | Annual Change: {annual_change}%")
        print(
            f"     Operating CF: ${operating_cf:,} | Profit Margin: {profit_margins} | ROA: {roa}"
        )
        print()

    # Show scoring breakdown for top stock
    if len(scored_stocks) > 0:
        top_stock = scored_stocks[0]
        print(f"=== Detailed Scoring for Top Stock: {top_stock['ticker']} ===")

        scoring_fields = [
            "operatingCashflow_score",
            "freeCashflow_score",
            "ebitda_score",
            "profitMargins_score",
            "revenueGrowth_score",
            "ebitdaMargins_score",
            "operatingMargins_score",
            "returnOnAssets_score",
            "returnOnEquity_score",
        ]

        for field in scoring_fields:
            if field in top_stock:
                metric_name = field.replace("_score", "")
                score = top_stock[field]
                value = top_stock.get(metric_name, "N/A")
                print(f"  {metric_name}: {value} (Rank: {score})")
else:
    print("❌ No stocks were scored (no recommended stocks found)")

# Compare with manual DataFrame approach from notebook
if "df" in locals() and len(df) > 0:
    print("\n=== Comparison with Notebook Implementation ===")
    print(f"Notebook approach found: {len(df)} stocks")
    print(f"Method approach found: {len(scored_stocks)} stocks")

    if len(scored_stocks) > 0 and len(df) > 0:
        # Compare top stock from each approach
        notebook_top = df.index[0]  # Best from notebook (lowest total_score)
        method_top = scored_stocks[0]["ticker"]  # Best from method

        print(f"Notebook top stock: {notebook_top}")
        print(f"Method top stock: {method_top}")
        print(f"Results match: {notebook_top == method_top}")

In [ ]:
# Run the #get_recommendations_with_scores
fas_full = await FundamentalStockAnalyzer.create()
print("=== Processing data from S&P 500 ===")
recommended_stocks = await fas_full.get_recommendations_with_scores(max_concurrent=10)

print("\n=== Top 10 Recommended Stocks (Best Scores) ===")
for i, stock in enumerate(recommended_stocks[:10], 1):
    ticker = stock["ticker"]
    name = stock.get("shortName", "N/A")
    total_score = stock["total_score"]
    annual_change = stock.get("annualChangePercent", "N/A")

    # Show some key financial metrics
    operating_cf = stock.get("operatingCashflow", "N/A")
    profit_margins = stock.get("profitMargins", "N/A")
    roa = stock.get("returnOnAssets", "N/A")

    print(f"{i:2d}. {ticker} - {name}")
    print(f"     Total Score: {total_score} | Annual Change: {annual_change}%")
    print(
        f"     Operating CF: ${operating_cf:,} | Profit Margin: {profit_margins} | ROA: {roa}"
    )
    print()